In [1]:
pip install pyod


     -------------------------------------- 150.0/150.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pyod: filename=pyod-1.0.9-py3-none-any.whl size=184163 sha256=14df8a73f358f22318648196d4aab7b637943398407db7800d8e6a40305ef9ba
  Stored in directory: c:\users\alham\appdata\local\pip\cache\wheels\8a\a9\b2\5db1f8c7dce5b24cee7c7254d162261df6bd85231a51867f1c
Successfully built pyod
Note: you may need to restart the kernel to use updated packages.


In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import median_abs_deviation
from pyod.models.mad import MAD
from sklearn.neighbors import LocalOutlierFactor
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
naamsestraat62 = pd.read_csv("C:\\Alhamd\\Belgium Study Material\\Modern Data Analytics\\Noise Data\\export_42\\csv_results_42_255441_mp-03-naamsestraat-62-taste.csv")
calvariekapel = pd.read_csv("C:\\Alhamd\\Belgium Study Material\\Modern Data Analytics\\Noise Data\\export_42\\csv_results_42_255442_mp-05-calvariekapel-ku-leuven.csv")
paarkstraat2 = pd.read_csv("C:\\Alhamd\\Belgium Study Material\\Modern Data Analytics\\Noise Data\\export_42\\csv_results_42_255443_mp-06-parkstraat-2-la-filosovia.csv")
naamsestraat81 = pd.read_csv("C:\\Alhamd\\Belgium Study Material\\Modern Data Analytics\\Noise Data\\export_42\\csv_results_42_255444_mp-07-naamsestraat-81.csv")


In [5]:
naamsestraat62_clean = naamsestraat62.dropna()

In [6]:
def categorize_columns(df, column_names, labels, categorization_names):
    new_df = df.copy()  # Create a copy of the DataFrame
    
    for column_name, categorization_name in zip(column_names, categorization_names):
        new_df[categorization_name] = 'Unknown'  # Create a new column with default values as 'Unknown'

        # Calculate quartiles
        quartiles = np.percentile(df[column_name], [25, 75])
        lower_threshold = quartiles[0]
        upper_threshold = quartiles[1]

        # Apply categorization based on quartiles
        new_df.loc[df[column_name] < lower_threshold, categorization_name] = labels[0]
        new_df.loc[(df[column_name] >= lower_threshold) & (df[column_name] <= upper_threshold), categorization_name] = labels[1]
        new_df.loc[df[column_name] > upper_threshold, categorization_name] = labels[2]

    return new_df

In [7]:
column_names = ['lamax', 'laeq', 'lceq', 'lcpeak']
labels = ['Low', 'Moderate', 'High']
categorization_names = ['lamax_category', 'laeq_category', 'lceq_category', 'lcpeak_category']

categorized_naamsestraat62 = categorize_columns(naamsestraat62_clean, column_names, labels, categorization_names)
categorized_naamsestraat62.head()

,object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,lcpeak_unit,lamax_category,laeq_category,lceq_category,lcpeak_category
0,255441,MP 03: Naamsestraat 62 Taste,01/01/2022 00:00:00.349,87.6,dB(A),82.7,dB(A),83.61,dB(C),97.17,dB(C),High,High,High,High
1,255441,MP 03: Naamsestraat 62 Taste,01/01/2022 00:00:01.349,84.5,dB(A),83.1,dB(A),84.42,dB(C),96.41,dB(C),High,High,High,High
2,255441,MP 03: Naamsestraat 62 Taste,01/01/2022 00:00:02.349,84.8,dB(A),82.7,dB(A),84.19,dB(C),96.24,dB(C),High,High,High,High
3,255441,MP 03: Naamsestraat 62 Taste,01/01/2022 00:00:03.349,81.9,dB(A),79.3,dB(A),81.08,dB(C),94.03,dB(C),High,High,High,High
4,255441,MP 03: Naamsestraat 62 Taste,01/01/2022 00:00:04.349,78.3,dB(A),76.0,dB(A),77.12,dB(C),89.81,dB(C),High,High,High,High


In [8]:
# Assuming your dataset is stored in a pandas DataFrame called 'df'
column_names = ['lamax', 'laeq', 'lceq', 'lcpeak']
labels = ['Low', 'Moderate', 'High']
categorization_names = ['lamax_category', 'laeq_category', 'lceq_category', 'lcpeak_category']

categorized_paarkstraat2 = categorize_columns(paarkstraat2, column_names, labels, categorization_names)

In [9]:
# Assuming your dataset is stored in a pandas DataFrame called 'df'
column_names = ['lamax', 'laeq', 'lceq', 'lcpeak']
labels = ['Low', 'Moderate', 'High']
categorization_names = ['lamax_category', 'laeq_category', 'lceq_category', 'lcpeak_category']

categorized_calveriekapel = categorize_columns(calvariekapel, column_names, labels, categorization_names)

In [10]:
# Assuming your dataset is stored in a pandas DataFrame called 'df'
column_names = ['lamax', 'laeq', 'lceq', 'lcpeak']
labels = ['Low', 'Moderate', 'High']
categorization_names = ['lamax_category', 'laeq_category', 'lceq_category', 'lcpeak_category']

categorized_naamsestraat81 = categorize_columns(naamsestraat81, column_names, labels, categorization_names)

In [11]:
def separate_timestamp(df, timestamp_column):
    df[timestamp_column] = pd.to_datetime(df[timestamp_column])
    df['day'] = df[timestamp_column].dt.day
    df['month'] = df[timestamp_column].dt.month
    df['week'] = df[timestamp_column].dt.week
    df['hour'] = df[timestamp_column].dt.hour
    df['minute'] = df[timestamp_column].dt.minute
    df['date']= df[timestamp_column].dt.date
    return df


In [12]:
categorized_naamsestraat62 = separate_timestamp(categorized_naamsestraat62, 'result_timestamp')
categorized_naamsestraat81 = separate_timestamp(categorized_naamsestraat81, 'result_timestamp')
categorized_calveriekapel = separate_timestamp(categorized_calveriekapel, 'result_timestamp')
categorized_paarkstraat2 = separate_timestamp(categorized_paarkstraat2, 'result_timestamp')

C:\Users\alham\AppData\Local\Temp\ipykernel_21584\4271267272.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['week'] = df[timestamp_column].dt.week
C:\Users\alham\AppData\Local\Temp\ipykernel_21584\4271267272.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['week'] = df[timestamp_column].dt.week
C:\Users\alham\AppData\Local\Temp\ipykernel_21584\4271267272.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['week'] = df[timestamp_column].dt.week
C:\Users\alham\AppData\Local\Temp\ipykernel_21584\4271267272.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['week'] = df[timestamp_column].dt.week


In [13]:
merged_datasets_all_locations = df_combined = pd.concat([categorized_naamsestraat62, categorized_naamsestraat81, categorized_calveriekapel, categorized_paarkstraat2], ignore_index=True)

In [14]:
def calculate_hourly_averages(df, hour_column, columns_to_average):
    # Group the data by hour and calculate the average for desired columns
    hourly_avg = df.groupby(hour_column).agg({col: 'mean' for col in columns_to_average}).reset_index()
    
    # Rename the columns for the hourly averages
    hourly_avg.rename(columns={col: 'hourly_' + col + '_avg' for col in columns_to_average}, inplace=True)
    
    # Merge the hourly averages back into the original dataset
    df = pd.merge(df, hourly_avg, on=hour_column, how='left')
    
    return df

In [17]:
# Specify the hour column and columns to average
hour_column = 'hour'
columns_to_average = ['lamax', 'lcpeak', 'laeq', 'lceq']

# Call the function to calculate hourly averages
merged_datasets_all_locations_agg_hour = calculate_hourly_averages(merged_datasets_all_locations, hour_column, columns_to_average)

In [18]:
merged_datasets_all_locations_agg_hour.head()

,object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,...,day,month,week,hour,minute,date,hourly_lamax_avg,hourly_lcpeak_avg,hourly_laeq_avg,hourly_lceq_avg
0,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:00.349,87.6,dB(A),82.7,dB(A),83.61,dB(C),97.17,...,1,1,52,0,0,2022-01-01,44.077798,64.959602,42.129398,52.49727
1,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:01.349,84.5,dB(A),83.1,dB(A),84.42,dB(C),96.41,...,1,1,52,0,0,2022-01-01,44.077798,64.959602,42.129398,52.49727
2,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:02.349,84.8,dB(A),82.7,dB(A),84.19,dB(C),96.24,...,1,1,52,0,0,2022-01-01,44.077798,64.959602,42.129398,52.49727
3,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:03.349,81.9,dB(A),79.3,dB(A),81.08,dB(C),94.03,...,1,1,52,0,0,2022-01-01,44.077798,64.959602,42.129398,52.49727
4,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:04.349,78.3,dB(A),76.0,dB(A),77.12,dB(C),89.81,...,1,1,52,0,0,2022-01-01,44.077798,64.959602,42.129398,52.49727


In [ ]:
# Call the function to calculate hourly averages
merged_datasets_all_locations_agg_hour = calculate_hourly_averages(merged_datasets_all_locations, hour_column, columns_to_average)

In [20]:
merged_datasets_all_locations_agg_hour.head()

,object_id,description,result_timestamp,lamax,lamax_unit,laeq,laeq_unit,lceq,lceq_unit,lcpeak,...,month,week,hour,minute,date,hourly_lamax_avg,hourly_lcpeak_avg,hourly_laeq_avg,hourly_lceq_avg,location
0,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:00.349,87.6,dB(A),82.7,dB(A),83.61,dB(C),97.17,...,1,52,0,0,2022-01-01,44.077798,64.959602,42.129398,52.49727,Naamsestraat 62
1,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:01.349,84.5,dB(A),83.1,dB(A),84.42,dB(C),96.41,...,1,52,0,0,2022-01-01,44.077798,64.959602,42.129398,52.49727,Naamsestraat 62
2,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:02.349,84.8,dB(A),82.7,dB(A),84.19,dB(C),96.24,...,1,52,0,0,2022-01-01,44.077798,64.959602,42.129398,52.49727,Naamsestraat 62
3,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:03.349,81.9,dB(A),79.3,dB(A),81.08,dB(C),94.03,...,1,52,0,0,2022-01-01,44.077798,64.959602,42.129398,52.49727,Naamsestraat 62
4,255441,MP 03: Naamsestraat 62 Taste,2022-01-01 00:00:04.349,78.3,dB(A),76.0,dB(A),77.12,dB(C),89.81,...,1,52,0,0,2022-01-01,44.077798,64.959602,42.129398,52.49727,Naamsestraat 62


In [19]:
merged_datasets_all_locations_agg_hour['location'] = ''

# Apply the conditions and update 'location' accordingly
merged_datasets_all_locations_agg_hour.loc[merged_datasets_all_locations_agg_hour['description'] == 'MP 03: Naamsestraat 62 Taste', 'location'] = 'Naamsestraat 62'
merged_datasets_all_locations_agg_hour.loc[merged_datasets_all_locations_agg_hour['description'] == 'MP 07: Naamsestraat 81', 'location'] = 'Naamsestraat 81'
merged_datasets_all_locations_agg_hour.loc[merged_datasets_all_locations_agg_hour['description'] == 'MP 05: Calvariekapel KU Leuven', 'location'] = 'Calveriekapel'
merged_datasets_all_locations_agg_hour.loc[merged_datasets_all_locations_agg_hour['description'] == 'MP 06: Parkstraat 2 La Filosovia', 'location'] = 'Parkstraat 2'


In [21]:
coordinates = {
    'Naamsestraat 62': (50.875809, 4.700110),
    'Calveriekapel': (50.8745267, 4.6999168),
    'Parkstraat 2': (50.8741177, 4.7000138),
    'Naamsestraat 81': (50.8738250, 4.7001178)
}

merged_datasets_all_locations_agg_hour['lat'] = merged_datasets_all_locations_agg_hour['location'].map(lambda x: coordinates[x][0])
merged_datasets_all_locations_agg_hour['long'] = merged_datasets_all_locations_agg_hour['location'].map(lambda x: coordinates[x][1])

In [22]:
location_data = {
    'Location': ['Naamsestraat 62', 'Calvariekapel', 'Parkstraat 2', 'Naamsestraat 81']
}

In [23]:
# Filter the data for a specific range of 30 days
#start_date = merged_datasets_all_locations_agg_hour['date'].min()
#end_date = start_date + pd.DateOffset(days=29)  # 30 days from the start date
#filtered_data = merged_datasets_all_locations_agg_hour[(merged_datasets_all_locations_agg_hour['date'] >= start_date) & (merged_datasets_all_locations_agg_hour['date'] <= end_date)]


C:\Users\alham\AppData\Local\Temp\ipykernel_21584\1031569789.py:4: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior. In a future version these will be considered non-comparable. Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  filtered_data = merged_datasets_all_locations_agg_hour[(merged_datasets_all_locations_agg_hour['date'] >= start_date) & (merged_datasets_all_locations_agg_hour['date'] <= end_date)]


In [40]:
# Aggregate the data by location and hour, taking the average of 'hourly_lamax_avg'
aggregated_data = merged_datasets_all_locations_agg_hour.groupby(['location','date', 'hour', 'long', 'lat']).agg({'hourly_lamax_avg': 'mean'}).reset_index()

In [41]:
aggregated_data_df = pd.DataFrame(aggregated_data)

In [42]:
aggregated_data_df.tail()

,location,date,hour,long,lat,hourly_lamax_avg
2959,Parkstraat 2,2022-12-01,19,4.700014,50.874118,55.951113
2960,Parkstraat 2,2022-12-01,20,4.700014,50.874118,54.874266
2961,Parkstraat 2,2022-12-01,21,4.700014,50.874118,52.910926
2962,Parkstraat 2,2022-12-01,22,4.700014,50.874118,51.151528
2963,Parkstraat 2,2022-12-01,23,4.700014,50.874118,48.314696


In [43]:
aggregated_data_df.head()

,location,date,hour,long,lat,hourly_lamax_avg
0,Calveriekapel,2022-01-01,0,4.699917,50.874527,44.077798
1,Calveriekapel,2022-01-01,1,4.699917,50.874527,41.325689
2,Calveriekapel,2022-01-01,2,4.699917,50.874527,40.165396
3,Calveriekapel,2022-01-01,3,4.699917,50.874527,39.346337
4,Calveriekapel,2022-01-01,4,4.699917,50.874527,39.606143


In [83]:
import datetime
selected_naamsestraat62 = aggregated_data_df[aggregated_data_df['location'] == 'Naamsestraat 62']
selected_naamsestraat62_date = selected_naamsestraat62[selected_naamsestraat62['date'] == datetime.date(2022, 1, 1)]
selected_naamsestraat62_hour = selected_naamsestraat62_date[selected_naamsestraat62_date['hour'] == 0] 


selected_naamsestraat81 = aggregated_data_df[aggregated_data_df['location'] == 'Naamsestraat 81']
selected_naamsestraat81_date = selected_naamsestraat81[selected_naamsestraat81['date'] == datetime.date(2022, 1, 1)]
selected_naamsestraat81_hour = selected_naamsestraat81_date[selected_naamsestraat81_date['hour'] == 0] 


selected_calveriekapel = aggregated_data_df[aggregated_data_df['location'] == 'Calveriekapel']
selected_calveriekapel_date = selected_calveriekapel[selected_calveriekapel['date'] == datetime.date(2022, 1, 1)]
selected_calveriekapel_hour = selected_calveriekapel_date[selected_calveriekapel_date['hour'] == 0] 


selected_parkstraat2 = aggregated_data_df[aggregated_data_df['location'] == 'Parkstraat 2']
selected_parkstraat2_date = selected_parkstraat2[selected_parkstraat2['date'] == datetime.date(2022, 1, 1)]
selected_parkstraat2_hour = selected_parkstraat2_date[selected_parkstraat2_date['hour'] == 0] 

selected_parkstraat2_hour.hourly_lamax_avg

2232    44.077798
Name: hourly_lamax_avg, dtype: float64

In [92]:
#Lamax

import plotly.graph_objects as go

fig = go.Figure()

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.875809],
        lon=[4.700110],
        z=selected_naamsestraat62_hour.hourly_lamax_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.875809],
        lon=[4.700110],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Naamsestraat 62",
        showlegend = False
    )
)

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.8738250],
        lon=[4.7001178],
        z=selected_naamsestraat81_hour.hourly_lamax_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.8738250],
        lon=[4.7001178],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Naamsestraat 81",
        showlegend = False
    )
)

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.8745267],
        lon=[4.6999168],
        z=selected_calveriekapel_hour.hourly_lamax_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.8745267],
        lon=[4.6999168],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Calveriekapel",
        showlegend = False
    )
)

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.8741177],
        lon=[4.7000138],
        z=selected_parkstraat2_hour.hourly_lamax_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.8741177],
        lon=[4.7000138],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Parkstraat 2",
        showlegend = False
    )
)

fig.update_layout(
    mapbox_style="stamen-terrain",
    mapbox_center={"lon": 180},
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    mapbox={
        'zoom': 15,
        'center': {'lon': aggregated_data.long.mean(), 'lat': aggregated_data.lat.mean()}
    }
)

fig.show()


In [97]:
# Aggregate the data by location and hour, taking the average of 'hourly_lamax_avg'
aggregated_data_lcpeak = merged_datasets_all_locations_agg_hour.groupby(['location','date', 'hour', 'long', 'lat']).agg({'hourly_lcpeak_avg': 'mean'}).reset_index()
aggregated_data_lcpeak_df = pd.DataFrame(aggregated_data_lcpeak)
aggregated_data_lcpeak_df.tail()

,location,date,hour,long,lat,hourly_lcpeak_avg
2959,Parkstraat 2,2022-12-01,19,4.700014,50.874118,74.319708
2960,Parkstraat 2,2022-12-01,20,4.700014,50.874118,73.380508
2961,Parkstraat 2,2022-12-01,21,4.700014,50.874118,71.815353
2962,Parkstraat 2,2022-12-01,22,4.700014,50.874118,70.108965
2963,Parkstraat 2,2022-12-01,23,4.700014,50.874118,67.631602


In [94]:
# Aggregate the data by location and hour, taking the average of 'hourly_lamax_avg'
aggregated_data_laeq = merged_datasets_all_locations_agg_hour.groupby(['location','date', 'hour', 'long', 'lat']).agg({'hourly_laeq_avg': 'mean'}).reset_index()
aggregated_data_laeq_df = pd.DataFrame(aggregated_data_laeq)
aggregated_data_laeq_df.tail()


,location,date,hour,long,lat,hourly_laeq_avg
2959,Parkstraat 2,2022-12-01,19,4.700014,50.874118,54.274191
2960,Parkstraat 2,2022-12-01,20,4.700014,50.874118,53.165918
2961,Parkstraat 2,2022-12-01,21,4.700014,50.874118,51.115490
2962,Parkstraat 2,2022-12-01,22,4.700014,50.874118,49.231101
2963,Parkstraat 2,2022-12-01,23,4.700014,50.874118,46.200029


In [95]:
# Aggregate the data by location and hour, taking the average of 'hourly_lamax_avg'
aggregated_data_lceq = merged_datasets_all_locations_agg_hour.groupby(['location','date', 'hour', 'long', 'lat']).agg({'hourly_lceq_avg': 'mean'}).reset_index()
aggregated_data_lceq_df = pd.DataFrame(aggregated_data_lceq)
aggregated_data_lceq_df.tail()


,location,date,hour,long,lat,hourly_lceq_avg
2959,Parkstraat 2,2022-12-01,19,4.700014,50.874118,62.201645
2960,Parkstraat 2,2022-12-01,20,4.700014,50.874118,61.246174
2961,Parkstraat 2,2022-12-01,21,4.700014,50.874118,59.658374
2962,Parkstraat 2,2022-12-01,22,4.700014,50.874118,57.734489
2963,Parkstraat 2,2022-12-01,23,4.700014,50.874118,55.003128


In [104]:
#lcpeak

selected_naamsestraat62_lcpeak_df = aggregated_data_lcpeak_df[aggregated_data_lcpeak_df['location'] == 'Naamsestraat 62']
selected_naamsestraat62_date_lcpeak_df = selected_naamsestraat62_lcpeak_df[selected_naamsestraat62_lcpeak_df['date'] == datetime.date(2022, 1, 1)]
selected_naamsestraat62_hour_lcpeak_df = selected_naamsestraat62_date_lcpeak_df[selected_naamsestraat62_date_lcpeak_df['hour'] == 0] 


selected_naamsestraat81_lcpeak_df = aggregated_data_lcpeak_df[aggregated_data_lcpeak_df['location'] == 'Naamsestraat 81']
selected_naamsestraat81_date_lcpeak_df = selected_naamsestraat81_lcpeak_df[selected_naamsestraat81_lcpeak_df['date'] == datetime.date(2022, 1, 1)]
selected_naamsestraat81_hour_lcpeak_df = selected_naamsestraat81_date_lcpeak_df[selected_naamsestraat81_date_lcpeak_df['hour'] == 0] 


selected_calveriekapel_lcpeak_df = aggregated_data_lcpeak_df[aggregated_data_lcpeak_df['location'] == 'Calveriekapel']
selected_calveriekapel_date_lcpeak_df = selected_calveriekapel_lcpeak_df[selected_calveriekapel_lcpeak_df['date'] == datetime.date(2022, 1, 1)]
selected_calveriekapel_hour_lcpeak_df = selected_calveriekapel_date_lcpeak_df[selected_calveriekapel_date_lcpeak_df['hour'] == 0] 


selected_parkstraat2_lcpeak_df = aggregated_data_lcpeak_df[aggregated_data_lcpeak_df['location'] == 'Parkstraat 2']
selected_parkstraat2_date_lcpeak_df = selected_parkstraat2_lcpeak_df[selected_parkstraat2_lcpeak_df['date'] == datetime.date(2022, 1, 1)]
selected_parkstraat2_hour_lcpeak_df = selected_parkstraat2_date_lcpeak_df[selected_parkstraat2_date_lcpeak_df['hour'] == 0] 

selected_parkstraat2_hour_lcpeak_df.hourly_lcpeak_avg

2232    64.959602
Name: hourly_lcpeak_avg, dtype: float64

In [111]:
#laeq

selected_naamsestraat62_laeq_df = aggregated_data_laeq_df[aggregated_data_laeq_df['location'] == 'Naamsestraat 62']
selected_naamsestraat62_date_laeq_df = selected_naamsestraat62_laeq_df[selected_naamsestraat62_laeq_df['date'] == datetime.date(2022, 1, 1)]
selected_naamsestraat62_hour_laeq_df = selected_naamsestraat62_date_laeq_df[selected_naamsestraat62_date_laeq_df['hour'] == 0]

selected_naamsestraat81_laeq_df = aggregated_data_laeq_df[aggregated_data_laeq_df['location'] == 'Naamsestraat 81']
selected_naamsestraat81_date_laeq_df = selected_naamsestraat81_laeq_df[selected_naamsestraat81_laeq_df['date'] == datetime.date(2022, 1, 1)]
selected_naamsestraat81_hour_laeq_df = selected_naamsestraat81_date_laeq_df[selected_naamsestraat81_date_laeq_df['hour'] == 0]

selected_calveriekapel_laeq_df = aggregated_data_laeq_df[aggregated_data_laeq_df['location'] == 'Calveriekapel']
selected_calveriekapel_date_laeq_df = selected_calveriekapel_laeq_df[selected_calveriekapel_laeq_df['date'] == datetime.date(2022, 1, 1)]
selected_calveriekapel_hour_laeq_df = selected_calveriekapel_date_laeq_df[selected_calveriekapel_date_laeq_df['hour'] == 0]

selected_parkstraat2_laeq_df = aggregated_data_laeq_df[aggregated_data_laeq_df['location'] == 'Parkstraat 2']
selected_parkstraat2_date_laeq_df = selected_parkstraat2_laeq_df[selected_parkstraat2_laeq_df['date'] == datetime.date(2022, 1, 1)]
selected_parkstraat2_hour_laeq_df = selected_parkstraat2_date_laeq_df[selected_parkstraat2_date_laeq_df['hour'] == 0]

selected_parkstraat2_hour_laeq_df.hourly_laeq_avg

2232    42.129398
Name: hourly_laeq_avg, dtype: float64

In [115]:
selected_naamsestraat62_lceq_df = aggregated_data_lceq_df[aggregated_data_lceq_df['location'] == 'Naamsestraat 62']
selected_naamsestraat62_date_lceq_df = selected_naamsestraat62_lceq_df[selected_naamsestraat62_lceq_df['date'] == datetime.date(2022, 1, 1)]
selected_naamsestraat62_hour_lceq_df = selected_naamsestraat62_date_lceq_df[selected_naamsestraat62_date_lceq_df['hour'] == 0]

selected_naamsestraat81_lceq_df = aggregated_data_lceq_df[aggregated_data_lceq_df['location'] == 'Naamsestraat 81']
selected_naamsestraat81_date_lceq_df = selected_naamsestraat81_lceq_df[selected_naamsestraat81_lceq_df['date'] == datetime.date(2022, 1, 1)]
selected_naamsestraat81_hour_lceq_df = selected_naamsestraat81_date_lceq_df[selected_naamsestraat81_date_lceq_df['hour'] == 0]

selected_calveriekapel_lceq_df = aggregated_data_lceq_df[aggregated_data_lceq_df['location'] == 'Calveriekapel']
selected_calveriekapel_date_lceq_df = selected_calveriekapel_lceq_df[selected_calveriekapel_lceq_df['date'] == datetime.date(2022, 1, 1)]
selected_calveriekapel_hour_lceq_df = selected_calveriekapel_date_lceq_df[selected_calveriekapel_date_lceq_df['hour'] == 0]

selected_parkstraat2_lceq_df = aggregated_data_lceq_df[aggregated_data_lceq_df['location'] == 'Parkstraat 2']
selected_parkstraat2_date_lceq_df = selected_parkstraat2_lceq_df[selected_parkstraat2_lceq_df['date'] == datetime.date(2022, 1, 1)]
selected_parkstraat2_hour_lceq_df = selected_parkstraat2_date_lceq_df[selected_parkstraat2_date_lceq_df['hour'] == 0]

selected_parkstraat2_hour_lceq_df.hourly_lceq_avg

2232    52.49727
Name: hourly_lceq_avg, dtype: float64

In [118]:
#Lcpeak

import plotly.graph_objects as go

fig = go.Figure()

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.875809],
        lon=[4.700110],
        z=selected_naamsestraat62_hour_lcpeak_df.hourly_lcpeak_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.875809],
        lon=[4.700110],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Naamsestraat 62",
        showlegend = False
    )
)

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.8738250],
        lon=[4.7001178],
        z=selected_naamsestraat81_hour_lcpeak_df.hourly_lcpeak_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.8738250],
        lon=[4.7001178],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Naamsestraat 81",
        showlegend = False
    )
)

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.8745267],
        lon=[4.6999168],
        z=selected_calveriekapel_hour_lcpeak_df.hourly_lcpeak_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.8745267],
        lon=[4.6999168],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Calveriekapel",
        showlegend = False
    )
)

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.8741177],
        lon=[4.7000138],
        z=selected_parkstraat2_hour_lcpeak_df.hourly_lcpeak_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.8741177],
        lon=[4.7000138],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Parkstraat 2",
        showlegend = False
    )
)

fig.update_layout(
    mapbox_style="stamen-terrain",
    mapbox_center={"lon": 180},
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    mapbox={
        'zoom': 15,
        'center': {'lon': aggregated_data.long.mean(), 'lat': aggregated_data.lat.mean()}
    }
)

fig.show()


In [120]:
#Laeq

import plotly.graph_objects as go

fig = go.Figure()

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.875809],
        lon=[4.700110],
        z=selected_naamsestraat62_hour_laeq_df.hourly_laeq_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.875809],
        lon=[4.700110],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Naamsestraat 62",
        showlegend = False
    )
)

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.8738250],
        lon=[4.7001178],
        z=selected_naamsestraat81_hour_laeq_df.hourly_laeq_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.8738250],
        lon=[4.7001178],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Naamsestraat 81",
        showlegend = False
    )
)

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.8745267],
        lon=[4.6999168],
        z=selected_calveriekapel_hour_laeq_df.hourly_laeq_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.8745267],
        lon=[4.6999168],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Calveriekapel",
        showlegend = False
    )
)

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.8741177],
        lon=[4.7000138],
        z=selected_parkstraat2_hour_laeq_df.hourly_laeq_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.8741177],
        lon=[4.7000138],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Parkstraat 2",
        showlegend = False
    )
)

fig.update_layout(
    mapbox_style="stamen-terrain",
    mapbox_center={"lon": 180},
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    mapbox={
        'zoom': 15,
        'center': {'lon': aggregated_data.long.mean(), 'lat': aggregated_data.lat.mean()}
    }
)

fig.show()


In [121]:
#Lceq

import plotly.graph_objects as go

fig = go.Figure()

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.875809],
        lon=[4.700110],
        z=selected_naamsestraat62_hour_lceq_df.hourly_lceq_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.875809],
        lon=[4.700110],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Naamsestraat 62",
        showlegend = False
    )
)

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.8738250],
        lon=[4.7001178],
        z=selected_naamsestraat81_hour_lceq_df.hourly_lceq_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.8738250],
        lon=[4.7001178],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Naamsestraat 81",
        showlegend = False
    )
)

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.8745267],
        lon=[4.6999168],
        z=selected_calveriekapel_hour_lceq_df.hourly_lceq_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.8745267],
        lon=[4.6999168],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Calveriekapel",
        showlegend = False
    )
)

# Add density mapbox trace
fig.add_trace(
    go.Densitymapbox(
        lat=[50.8741177],
        lon=[4.7000138],
        z=selected_parkstraat2_hour_lceq_df.hourly_lceq_avg,
        radius=10
    )
)

# Add scattermapbox trace for individual markers
fig.add_trace(
    go.Scattermapbox(
        lat=[50.8741177],
        lon=[4.7000138],
        mode='markers',
        marker=dict(
            size=5,
            color='red',
            opacity=0.7
        ),
        hovertemplate="Parkstraat 2",
        showlegend = False
    )
)

fig.update_layout(
    mapbox_style="stamen-terrain",
    mapbox_center={"lon": 180},
    margin={"r": 0, "t": 0, "l": 0, "b": 0},
    mapbox={
        'zoom': 15,
        'center': {'lon': aggregated_data.long.mean(), 'lat': aggregated_data.lat.mean()}
    }
)

fig.show()